In [ ]:
# https://blog.myyellowroad.com/unsupervised-sentence-representation-with-deep-learning-104b90079a93

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)

In [5]:
#retrieve table titles from csv
data = pd.read_csv('./topic_modeling/titles.csv', delimiter='\t', error_bad_lines=True, header=None)

data.columns = ['id', 'title']
data.title = data.title.str.strip()
documents = data
documents['title'].replace('', np.nan, inplace=True)
documents = documents.astype(str)
print('checking if text is missing')
print(documents.isna().any())

documents.shape

checking if text is missing
id       False
title    False
dtype: bool


(46641, 2)

In [6]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import strip_numeric
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [7]:
np.random.seed(2018)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/grace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
stemmer = SnowballStemmer('english')
STOP_WORDS = list(gensim.parsing.preprocessing.STOPWORDS)
STOP_WORDS.extend(['table', 'legend'])

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
#     deacc=True removes punctuations
    for token in gensim.utils.simple_preprocess(text, deacc=True):
        if token not in STOP_WORDS and len(token)>1:
#             result.append(lemmatize_stemming(strip_numeric(token)))
            result.append(lemmatize_stemming(token))
    return result

In [9]:
%%time
#process the text, save the results as processed_docs
processed_docs = pd.DataFrame()
processed_docs = pd.concat([documents.id, documents.title.map(preprocess)], axis=1)

CPU times: user 12 s, sys: 56 ms, total: 12.1 s
Wall time: 12.1 s


In [10]:
processed_docs[:5]

,id,title
0,4105,"[baselin, characterist]"
1,4106,"[analysi, efficaci]"
2,4107,"[comparison, postop, carbohydr, antigen, level, surviv, espac, conoko, jaspac, trial]"
3,4108,"[pattern, diseas, relaps]"
4,4109,"[grade, advers, event, gemcitabin, gemcitabin, plus, capecitabin]"


In [12]:
dictionary = gensim.corpora.Dictionary(processed_docs.title)
print('size of the dictionary %d' %len(dictionary))
max_dic_size = len(dictionary)

size of the dictionary 12915


In [ ]:
#Building X and Y
num_words=2000
maxlen=20
tokenizer = Tokenizer(num_words = num_words, split=' ')
tokenizer.fit_on_texts(df['body'].values)
seqs = tokenizer.texts_to_sequences(df['body'].values)
seq = []
for i in seqs:
    seq+=i
    
X = []
Y = []
for i in tqdm(range(len(seq)-maxlen-1)):
    X.append(seq[i:i+maxlen])
    Y.append(seq[i+maxlen+1])
X = pd.DataFrame(X)
Y = pd.DataFrame(Y)
Y[0]=Y[0].astype('category')
Y =pd.get_dummies(Y)
#Buidling the network
embed_dim = 150
lstm_out = 128
batch_size= 128
model = Sequential()
model.add(Embedding(num_words, embed_dim,input_length = maxlen))
model.add(Bidirectional(LSTM(lstm_out)))
model.add(Dense(Y.shape[1],activation='softmax'))
adam = Adam(lr=0.001, beta_1=0.7, beta_2=0.99, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss = 'categorical_crossentropy', optimizer=adam)
model.summary()
print('fit')
model.fit(X, Y, batch_size =batch_size,validation_split=0.1, epochs = 5,  verbose = 1)

#Feature extraction
headlines = tokenizer.texts_to_sequences(data['headline'].values)
headlines = pad_sequences(headlines,maxlen=maxlen)
inp = model.input                                          
outputs = [model.layers[1].output]      
functor = K.function([inp]+ [K.learning_phase()], outputs )
x = functor([headlines, 1.])[0]

#classifier
X_train,y_train,X_test,y_test = x[msk],y[msk],x[~msk],y[~msk]
lr = LogisticRegression().fit(X_train,y_train)
lr.score(X_test,y_test)